# ДОМАШНЕЕ ЗАДАНИЕ № 2. Анализ поездок посредством Spark DataFrame API
---
**Дисциплина:** Методы обработки больших данных  
**Студент:** Голдышев Д.М. (goldyshev02@mail.ru)  
**Группа:** ИУ6-31М  

## Задача 3
---
### Цель
Проанализировать поездки с использованием **Spark DataFrame API**.

### Задачи
1. Для каждой станции определить среднее количество начала и завершения поездок:
   - За день
   - По времени суток: утро (06:00-11:59), день (12:00-17:59), вечер (18:00-23:59), ночь (00:00-05:59)
   - Отдельно для среды и воскресенья по временным диапазонам
2. Отобразить данные по времени суток в виде тепловой карты (HeatMapWithTime)

### Данные
- [Данные о поездках](https://s3.amazonaws.com/tripdata/JC-201902-citibike-tripdata.csv.zip)
- ~~Границы зон~~ (по заданию не нужны)

## Шаг 1. Импорт зависимостей и конфигурация
---

In [1]:
import os
import sys
import errno
import tempfile
from pathlib import Path

import csv
import io
import json
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StringType

import pandas as pd
import folium
from folium.plugins import HeatMapWithTime

# Путь к данным о поездках
DATA_PATH = "/app/data/bikeshare/JC-201902-citibike-tripdata.csv"

# Пути для сохранения результатов
OUTPUT_DIR = "."
OUTPUT_RESULTS_DAILY = f"{OUTPUT_DIR}/hw2_bikeshare_1_1_daily.csv"
OUTPUT_RESULTS_BY_PERIOD = f"{OUTPUT_DIR}/hw2_bikeshare_1_2_by_period.csv"
OUTPUT_RESULTS_WEDNESDAY = f"{OUTPUT_DIR}/hw2_bikeshare_1_3_wednesday.csv"
OUTPUT_RESULTS_SUNDAY = f"{OUTPUT_DIR}/hw2_bikeshare_1_3_sunday.csv"
OUTPUT_VISUALIZATION = f"{OUTPUT_DIR}/hw2_bikeshare_2_HeatMapWithTime.html"

### Проверка существования данных

In [2]:
def check_file(path: str):
    if not os.path.isfile(path):
        raise FileNotFoundError(f"{path}: файл не найден")
    try:
        with open(path, "r", encoding="utf-8") as f:
            f.read(1)  # пробное чтение
    except Exception as e:
        raise IOError(f"{path}: файл существует, но не читается: {e}")

try:
    check_file(DATA_PATH)
    print("Файлы, необходимые для работы, найдены и читаются.")
except Exception as e:
    print("Ошибка при проверке файлов:")
    print(e)
    raise SystemExit(1)

Файлы, необходимые для работы, найдены и читаются.


### Проверка возможности сохранения результатов

In [3]:
def ensure_output_path(path: str) -> Path:
    p = Path(path).expanduser()
    if p.exists() and p.is_dir():
        dir_path = p
    else:
        if str(p).endswith(os.sep) or (not p.exists() and p.suffix == ""):
            dir_path = p
        else:
            dir_path = p.parent
    if not str(dir_path):
        dir_path = Path(".")
    try:
        dir_path.mkdir(parents=True, exist_ok=True)
    except PermissionError as e:
        raise PermissionError(f"{dir_path}: нет прав на создание директории: {e}")
    except OSError as e:
        raise OSError(f"{dir_path}: не удалось создать директорию: {e}")
    if not os.access(dir_path, os.W_OK):
        raise PermissionError(f"{dir_path}: нет прав на запись в директорию")
    try:
        with tempfile.NamedTemporaryFile(dir=dir_path, delete=True) as tmp:
            tmp.write(b"test")
            tmp.flush()
    except OSError as e:
        raise OSError(f"{dir_path}: невозможно записать временный файл: {e}")
    return dir_path

try:
    ensure_output_path(OUTPUT_DIR)
    print("Можно сохранять результаты в указанные директории")
except Exception as e:
    print("Ошибка при проверке директорий для записи результатов:")
    print(e)
    raise SystemExit(1)

Можно сохранять результаты в указанные директории


## Шаг 2. Функция определения периода суток
---

In [4]:
def get_time_period(hour):
    """
    Определяет период суток по часу.
    Args:
        hour: час (0-23)
    Returns:
        строка: 'morning', 'afternoon', 'evening' или 'night'
    Периоды:
        - morning: 06:00-11:59
        - afternoon: 12:00-17:59
        - evening: 18:00-23:59
        - night: 00:00-05:59
    """
    if 6 <= hour <= 11:
        return "morning"
    elif 12 <= hour <= 17:
        return "afternoon"
    elif 18 <= hour <= 23:
        return "evening"
    else:
        return "night"

# Регистрируем функцию как UDF (User Defined Function) для использования в DataFrame
get_period_udf = F.udf(get_time_period, StringType())

## Шаг 3. Инициализация SparkSession
---

In [5]:
spark = SparkSession.builder \
    .appName("HW2_Bikeshare") \
    .master("local[*]") \
    .getOrCreate()
# Количество партиций при shuffle и настройки памяти указаны на уровне Docker-контейнера

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/10 22:01:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/10 22:01:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Шаг 4. Загрузка и предобработка данных
---

In [6]:
# Читаем CSV с автоматическим определением типов данных
df = spark.read.csv(DATA_PATH, header=True, inferSchema=True)

# Выводим схему для проверки
print("Схема данных:")
df.printSchema()
print(f"Всего записей: {df.count()}")

Схема данных:
root
 |-- tripduration: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- stoptime: timestamp (nullable = true)
 |-- start station id: integer (nullable = true)
 |-- start station name: string (nullable = true)
 |-- start station latitude: double (nullable = true)
 |-- start station longitude: double (nullable = true)
 |-- end station id: integer (nullable = true)
 |-- end station name: string (nullable = true)
 |-- end station latitude: double (nullable = true)
 |-- end station longitude: double (nullable = true)
 |-- bikeid: integer (nullable = true)
 |-- usertype: string (nullable = true)
 |-- birth year: integer (nullable = true)
 |-- gender: integer (nullable = true)

Всего записей: 18565


In [7]:
# Извлекаем нужные данные, парсим даты и добавляем вычисляемые колонки
df = df \
    .withColumn("start_ts", F.to_timestamp("starttime", "yyyy-MM-dd HH:mm:ss.SSSS")) \
    .withColumn("stop_ts", F.to_timestamp("stoptime", "yyyy-MM-dd HH:mm:ss.SSSS")) \
    .withColumn("start_hour", F.hour("start_ts")) \
    .withColumn("stop_hour", F.hour("stop_ts")) \
    .withColumn("start_date", F.to_date("start_ts")) \
    .withColumn("stop_date", F.to_date("stop_ts")) \
    .withColumn("day_of_week", F.dayofweek("start_ts"))

# Добавляем период суток через UDF
df = df.withColumn("start_period", get_period_udf("start_hour"))
df = df.withColumn("stop_period", get_period_udf("stop_hour"))

# Кэшируем DataFrame для повторного использования
df.cache()

# Проверяем результат
print("Частичный вывод полученных данных:")
print("(чтобы не растягивать таблицу, выводим только данные о начале поездки)")
df.select("start station name", "starttime", "start_ts", "start_hour", "start_period", "day_of_week").show(5, truncate=False)

Частичный вывод полученных данных:
(чтобы не растягивать таблицу, выводим только данные о начале поездки)


[Stage 5:>                                                          (0 + 1) / 1]

+------------------+-----------------------+-----------------------+----------+------------+-----------+
|start station name|starttime              |start_ts               |start_hour|start_period|day_of_week|
+------------------+-----------------------+-----------------------+----------+------------+-----------+
|Exchange Place    |2019-02-01 15:35:02.082|2019-02-01 15:35:02.082|15        |afternoon   |6          |
|Exchange Place    |2019-02-01 17:00:46.89 |2019-02-01 17:00:46.89 |17        |afternoon   |6          |
|Exchange Place    |2019-02-01 17:08:01.326|2019-02-01 17:08:01.326|17        |afternoon   |6          |
|Exchange Place    |2019-02-01 17:09:31.21 |2019-02-01 17:09:31.21 |17        |afternoon   |6          |
|Exchange Place    |2019-02-01 17:19:53.249|2019-02-01 17:19:53.249|17        |afternoon   |6          |
+------------------+-----------------------+-----------------------+----------+------------+-----------+
only showing top 5 rows



## Шаг 5. Подсчёт количества дней в датасете
---
Для расчёта средних значений нам нужно знать общее количество дней в датасете:
$$\text{Среднее} = \frac{\text{Общее количество поездок}}{\text{Количество дней в датасете}}$$

In [8]:
# Считаем количество уникальных дат начала поездок
total_days = df.select("start_date").distinct().count()

# Считаем количество сред (day_of_week = 4) в датасете
wednesday_days = df.filter(F.col("day_of_week") == 4).select("start_date").distinct().count()
# Считаем количество воскресений (day_of_week = 1) в датасете
sunday_days = df.filter(F.col("day_of_week") == 1).select("start_date").distinct().count()

print(f"Всего дней в датасете: {total_days}")
print(f"Из них сред: {wednesday_days}")
print(f"Из них воскресений: {sunday_days}")

# Сохраняем в переменные для дальнейшего использования в расчётах
# (преобразуем в float для корректного деления)
TOTAL_DAYS = float(total_days)
WEDNESDAY_DAYS = float(wednesday_days)
SUNDAY_DAYS = float(sunday_days)

# Понадобится для сравнения чисел с плавающей запятой
EPS = 1e-9

Всего дней в датасете: 28
Из них сред: 4
Из них воскресений: 4


### Дополнительно: подсчет станций
Это нужно для более удобного вывода данных, т.к. `show()` без параметров сокращает количество отображаемых строк.

In [9]:
STATIONS_COUNT = (
    df.select(F.col("start station name").alias("station"))
      .union(df.select(F.col("end station name").alias("station")))
).select("station").distinct().count()
print("Всего уникальных станций:", STATIONS_COUNT)

Всего уникальных станций: 52


## Шаг 6. Среднее количество поездок по станциям за день
---

In [10]:
# Группируем данные по названию начальной станции и считаем количество поездок.
starts_total = df \
    .groupBy("start station name") \
    .agg(F.count("*").alias("total_starts")) \
    .withColumnRenamed("start station name", "station_name")

# Аналогично для станций завершения поездок
ends_total = df \
    .groupBy("end station name") \
    .agg(F.count("*").alias("total_ends")) \
    .withColumnRenamed("end station name", "station_name")

# Объединяем две таблицы по названию станции (+ заполняем нулями, если станция не встречается как start или end)
daily_stats = starts_total.join(
    ends_total,
    on="station_name",
    how="outer"  # Полное внешнее объединение, чтобы не потерять станции
).fillna({"total_starts": 0, "total_ends": 0})

# Рассчитываем средние значения
daily_stats = daily_stats \
    .withColumn(
        "avg_starts_per_day",
        F.round(F.col("total_starts") / F.lit(TOTAL_DAYS), 2)  # F.lit() — литеральное значение
    ) \
    .withColumn(
        "avg_ends_per_day",
        F.round(F.col("total_ends") / F.lit(TOTAL_DAYS), 2)
    )

# Выбираем и сортируем (по названию станций) финальные колонки.
daily_stats = daily_stats.select(
    "station_name",
    "avg_starts_per_day",
    "avg_ends_per_day"
).orderBy("station_name")

# Вывод результатов
print("Среднее количество поездок за день:")
daily_stats.show(STATIONS_COUNT, truncate=False)

# Сохраняем результаты в CSV (т.к. данных совсем немного, то используем `toPandas()`)
daily_stats.toPandas().to_csv(OUTPUT_RESULTS_DAILY, index=False)
print(f"Результаты сохранены в {OUTPUT_RESULTS_DAILY}")

Среднее количество поездок за день:
+-----------------------+------------------+----------------+
|station_name           |avg_starts_per_day|avg_ends_per_day|
+-----------------------+------------------+----------------+
|5 Corners Library      |3.96              |3.18            |
|Astor Place            |8.14              |8.0             |
|Baldwin at Montgomery  |7.43              |6.64            |
|Bergen Ave             |6.18              |5.57            |
|Brunswick & 6th        |16.14             |14.29           |
|Brunswick St           |18.29             |16.14           |
|Christ Hospital        |3.68              |2.32            |
|City Hall              |17.75             |18.75           |
|Columbus Drive         |15.46             |15.29           |
|Communipaw & Berry Lane|1.86              |1.68            |
|Dey St                 |1.86              |3.54            |
|Dixon Mills            |13.07             |11.82           |
|Essex Light Rail       |10.04    

## Шаг 7. Среднее количество поездок по станциям по времени суток
---

### Выделяем общую функцию для расчета средних дней
Эта функция будет полезна как для выполнения текущего задания, так и для выполнения следующего задания.

In [11]:
PERIODS = ["morning", "afternoon", "evening", "night"]
# Формируем список сгенерированных обозначений начал и окончаний поездок в определенное время суток
PIVOT_VALUES = [
    col
    for p in PERIODS
    for col in (f"start_{p}", f"end_{p}")
]

def calculate_period_stats(df_source, num_days):
    """
    Рассчитывает среднее количество начала и завершения поездок
    по периодам суток для переданного набора данных.
    Args:
        df_source: набор данных, для которого нужно рассчитать среднее
        num_days: количество дней в датасете
    Returns:
        Набор данных со средними значениями по периодам суток
    """

    # Формируем данные в формате (станция, время суток, начало/конец поездки)
    df_period_with_kind = df_source \
        .select(
            F.col("start station name").alias("station_name"),
            F.col("start_period").alias("period"),
            F.lit("start").alias("kind")
        ) \
        .unionByName(
            df_source.select(
                F.col("end station name").alias("station_name"),
                F.col("stop_period").alias("period"),
                F.lit("end").alias("kind")
            )
        ) \
        .where(F.col("period").isNotNull())

    # Получаем количество начал и окончаний поездок для каждой станции 
    # в определенное время суток
    df_grouped = df_period_with_kind \
        .withColumn("kind_period", F.concat_ws("_", "kind", "period")) \
        .groupBy("station_name", "kind_period") \
        .agg(F.count("*").alias("cnt"))

    # Получаем по строке на каждую станцию и колонку на каждую пару "тип_период"
    period_stats = df_grouped \
        .groupBy("station_name") \
        .pivot("kind_period", PIVOT_VALUES) \
        .sum("cnt") \
        .fillna(0)

    # "Превращаем" абсолютные значения в среднее число поездок
    for col_name in PIVOT_VALUES:
        period_stats = period_stats.withColumn(
            col_name,
            F.round(F.col(col_name) / F.lit(num_days), 2)
        )

    # Упорядочиваем колонки и сортируем по названию станции
    period_stats = period_stats \
        .select(["station_name"] + PIVOT_VALUES) \
        .orderBy("station_name")

    return period_stats

### Вычисляем среднее количество поездок по времени суток за все дни

In [12]:
period_stats = calculate_period_stats(df, TOTAL_DAYS)

print("Среднее количество поездок по времени суток:")
period_stats.show(STATIONS_COUNT, truncate=False)

# Сохраняем результаты в CSV (т.к. данных совсем немного, то используем `toPandas()`)
period_stats.toPandas().to_csv(OUTPUT_RESULTS_BY_PERIOD, index=False)
print(f"Результаты сохранены в {OUTPUT_RESULTS_BY_PERIOD}")

Среднее количество поездок по времени суток:
+-----------------------+-------------+-----------+---------------+-------------+-------------+-----------+-----------+---------+
|station_name           |start_morning|end_morning|start_afternoon|end_afternoon|start_evening|end_evening|start_night|end_night|
+-----------------------+-------------+-----------+---------------+-------------+-------------+-----------+-----------+---------+
|5 Corners Library      |0.61         |0.32       |0.93           |1.82         |2.18         |1.04       |0.25       |0.0      |
|Astor Place            |5.11         |0.36       |2.25           |3.32         |0.61         |4.07       |0.18       |0.25     |
|Baldwin at Montgomery  |3.89         |0.79       |1.86           |1.71         |0.71         |4.0        |0.96       |0.14     |
|Bergen Ave             |2.68         |1.79       |1.71           |1.39         |1.68         |1.64       |0.11       |0.75     |
|Brunswick & 6th        |10.11        |1.93  

## Шаг 8. Среднее количество поездок по средам и воскресеньям
---

In [13]:
days_config = [
    (4, WEDNESDAY_DAYS, "средам", OUTPUT_RESULTS_WEDNESDAY), 
    (1, SUNDAY_DAYS, "воскресеньям", OUTPUT_RESULTS_SUNDAY),
]
for day_of_week, num_days, phrase, output_path in days_config:
    # Фильтруем по дню недели
    df_day = df.filter(F.col("day_of_week") == day_of_week)
    day_stats = calculate_period_stats(df_day, num_days)
    print(f"Среднее количество поездок по {phrase}:")
    day_stats.show(STATIONS_COUNT, truncate=False)
    # Сохраняем результаты в CSV (т.к. данных совсем немного, то используем `toPandas()`)
    day_stats.toPandas().to_csv(output_path, index=False)
    print(f"Результаты сохранены в {output_path}\n\n")

Среднее количество поездок по средам:
+-----------------------+-------------+-----------+---------------+-------------+-------------+-----------+-----------+---------+
|station_name           |start_morning|end_morning|start_afternoon|end_afternoon|start_evening|end_evening|start_night|end_night|
+-----------------------+-------------+-----------+---------------+-------------+-------------+-----------+-----------+---------+
|5 Corners Library      |0.5          |0.25       |0.0            |1.75         |2.25         |1.25       |0.25       |0.0      |
|Astor Place            |6.25         |0.5        |2.0            |4.0          |0.5          |4.25       |0.0        |0.25     |
|Baldwin at Montgomery  |6.25         |1.5        |0.0            |1.0          |0.25         |3.25       |1.0        |0.0      |
|Bergen Ave             |3.75         |2.0        |1.0            |1.5          |1.25         |1.75       |0.25       |0.75     |
|Brunswick & 6th        |13.25        |1.5        |3

## Шаг 9. Создание тепловой карты HeatMapWithTime
---

### Подготовка основного датасета

In [14]:
# Собираем координаты из обоих источников (т.к. обнаружены станции, на которых только заканчивали поездки)
start_coords = df.select(
    F.col("start station name").alias("station_name"),
    F.col("start station latitude").alias("latitude"),
    F.col("start station longitude").alias("longitude")
)
end_coords = df.select(
    F.col("end station name").alias("station_name"),
    F.col("end station latitude").alias("latitude"),
    F.col("end station longitude").alias("longitude")
)
stations_coords = start_coords.unionByName(end_coords).distinct()

# Объединяем координаты с полученной ранее статистикой
map_data = period_stats.join(
    stations_coords,
    on="station_name",
    how="inner"
)

# Конвертируем в pandas для работы с folium
map_df = map_data.toPandas()

# Проверяем, что в итоге получаем датасет с ожидаемым количеством станций
if len(map_df) != STATIONS_COUNT:
    print(f"Должно быть {STATIONS_COUNT} станций")
    raise SystemExit(1)

### Создание базовой карты

In [15]:
# Вычисляем центр карты как среднее координат всех станций
center_lat = float(map_df["latitude"].mean())
center_lon = float(map_df["longitude"].mean())
print(f"Центр карты: ({center_lat:.6f}, {center_lon:.6f})")

# Создаём базовую карту с помощью folium
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=13,
    tiles="cartodbpositron" # используем светлую тему
)

Центр карты: (40.724156, -74.050992)


### Подготовка данных для HeatMapWithTime

In [16]:
period_labels = [
    "06:00-11:59 (Утро)",
    "12:00-17:59 (День)", 
    "18:00-23:59 (Вечер)",
    "00:00-05:59 (Ночь)"
]

# Находим максимальную и минимальную (ненулевую) сумму для нормализации весов тепловой карты
max_total = 0.0
min_total = float('inf')
for period in PERIODS:
    start_col = f"start_{period}"
    end_col = f"end_{period}"
    period_sum = map_df[start_col] + map_df[end_col]
    period_max = float(period_sum.max())
    # Минимум среди ненулевых значений
    nonzero_values = period_sum[period_sum > 0]
    if len(nonzero_values) > 0:
        period_min = float(nonzero_values.min())
        if period_min < min_total:
            min_total = period_min
    if period_max > max_total:
        max_total = period_max

if max_total > EPS:
    print(f"Диапазон значений для периодов: {min_total:.2f} - {max_total:.2f}")
else:
    print(f"Ошибка (скорее всего)! Получили `max_total = {max_total}`, но в таком случае нам нечего отображать...")
    raise SystemExit(1)

# Формируем данные для HeatMapWithTime
heat_data = []
for period in PERIODS:
    start_col = f"start_{period}"
    end_col = f"end_{period}"
    layer = []
    stations_in_layer = 0
    stations_skipped = 0
    # Для текущего слоя ожидаем `[[lat, lon, weight], ...]`
    for _, row in map_df.iterrows():
        lat = float(row["latitude"])
        lon = float(row["longitude"])
        
        avg_starts = float(row[start_col])
        avg_ends = float(row[end_col])
        total = avg_starts + avg_ends

        # Пропускаем нулевые значения
        if total <= EPS:
            stations_skipped += 1
            continue
        
        # Используем абсолютную нормализацию
        normalized = total / max_total # Ранее проверили, что max_total > 0 => можем безопасно делить
        # Корень для усиления контраста низких значений.
        weight = float(pow(normalized, 0.5))
        layer.append([lat, lon, weight])
        stations_in_layer += 1
    
    heat_data.append(layer)
    print(f"Слой {period}: {stations_in_layer} точек (пропущено с нулём: {stations_skipped})")

Диапазон значений для периодов: 0.04 - 74.29
Слой morning: 51 точек (пропущено с нулём: 1)
Слой afternoon: 51 точек (пропущено с нулём: 1)
Слой evening: 52 точек (пропущено с нулём: 0)
Слой night: 50 точек (пропущено с нулём: 2)


In [17]:
# Добавляем HeatMapWithTime с расширенным градиентом
HeatMapWithTime(
    data=heat_data,
    index=period_labels,
    auto_play=True,
    max_opacity=0.9,
    min_opacity=0.3,
    radius=35,
    use_local_extrema=False, # нормализация "яркости" идёт по всему набору данных сразу, а не по каждому кадру отдельно
    # Используем кастомный расширенный градиент с большим количеством цветов для лучшего различия
    gradient={
        0.0: 'navy',
        0.15: 'blue',
        0.3: 'cyan',
        0.45: 'lime',
        0.6: 'yellow',
        0.75: 'orange',
        0.9: 'red',
        1.0: 'darkred'
    }
).add_to(m)

### Добавление маркеров с всплывающими окнами

In [18]:
# Вычисляем общую активность для размера маркеров
map_df["total_activity"] = (
    map_df["start_morning"] + map_df["end_morning"] +
    map_df["start_afternoon"] + map_df["end_afternoon"] +
    map_df["start_evening"] + map_df["end_evening"] +
    map_df["start_night"] + map_df["end_night"]
)
max_activity = float(map_df["total_activity"].max())
min_activity = float(map_df["total_activity"][map_df["total_activity"] > 0].min())

if max_activity > EPS:
    print(f"Диапазон значений для суммарной активности: {min_activity:.2f} - {max_activity:.2f}")
else:
    print(f"Ошибка (скорее всего)! Получили `max_activity = {max_activity}`, но в таком случае нам нечего отображать...")
    raise SystemExit(1)

for _, row in map_df.iterrows():
    start_morning = float(row['start_morning'])
    end_morning = float(row['end_morning'])
    start_afternoon = float(row['start_afternoon'])
    end_afternoon = float(row['end_afternoon'])
    start_evening = float(row['start_evening'])
    end_evening = float(row['end_evening'])
    start_night = float(row['start_night'])
    end_night = float(row['end_night'])
    total_act = float(row["total_activity"])

    # Отображаем всю информацию во всплывающем окне используя HTML-таблицу
    popup_html = f"""
    <div style="font-family: Arial; font-size: 12px; width: 300px;">
        <b>{row['station_name']}</b><br><br>
        <table style="width: 100%; border-collapse: collapse;">
            <tr style="background-color: #e0e0e0;">
                <th style="padding: 4px; text-align: left;">Period</th>
                <th style="padding: 4px;">Ср. число стартов</th>
                <th style="padding: 4px;">Ср. число окончаний</th>
                <th style="padding: 4px;">Ср. общее</th>
            </tr>
            <tr>
                <td style="padding: 4px;">Утро</td>
                <td style="padding: 4px; text-align: center;">{start_morning:.2f}</td>
                <td style="padding: 4px; text-align: center;">{end_morning:.2f}</td>
                <td style="padding: 4px; text-align: center;">{start_morning + end_morning:.2f}</td>
            </tr>
            <tr style="background-color: #f5f5f5;">
                <td style="padding: 4px;">День</td>
                <td style="padding: 4px; text-align: center;">{start_afternoon:.2f}</td>
                <td style="padding: 4px; text-align: center;">{end_afternoon:.2f}</td>
                <td style="padding: 4px; text-align: center;">{start_afternoon + end_afternoon:.2f}</td>
            </tr>
            <tr>
                <td style="padding: 4px;">Вечер</td>
                <td style="padding: 4px; text-align: center;">{start_evening:.2f}</td>
                <td style="padding: 4px; text-align: center;">{end_evening:.2f}</td>
                <td style="padding: 4px; text-align: center;">{start_evening + end_evening:.2f}</td>
            </tr>
            <tr style="background-color: #f5f5f5;">
                <td style="padding: 4px;">Ночь</td>
                <td style="padding: 4px; text-align: center;">{start_night:.2f}</td>
                <td style="padding: 4px; text-align: center;">{end_night:.2f}</td>
                <td style="padding: 4px; text-align: center;">{start_night + end_night:.2f}</td>
            </tr>
        </table>
    </div>
    """
    
    # Используем абсолютную нормализацию
    normalized_size = total_act / max_activity
    # Размер от 4 до 14 пикселей с корневой шкалой
    marker_size = 4 + pow(normalized_size, 0.5) * 10

    # Добавляем маркер
    folium.CircleMarker(
        location=[float(row["latitude"]), float(row["longitude"])],
        radius=marker_size,
        color="darkblue",
        fill=True,
        fillColor="blue",
        fillOpacity=0.7,
        popup=folium.Popup(popup_html, max_width=350)
    ).add_to(m)

Диапазон значений для суммарной активности: 0.04 - 186.47


### Отображение карты

In [19]:
display(m)

### Сохранение карты

In [20]:
m.save(OUTPUT_VISUALIZATION)
print(f"Карта сохранена: {OUTPUT_VISUALIZATION}")

Карта сохранена: ./hw2_bikeshare_2_HeatMapWithTime.html


## Выводы
---

В рамках задания был проведён анализ поездок на велосипеде с использованием **Spark DataFrame API**. Для всех станций рассчитано **среднее количество начала и завершения поездок** в разрезе различных временных периодов:
- **определено среднее количество поездок за день для каждой станции:** результаты показывают распределение нагрузки по станциям — от малоактивных точек до крупных;
- **выполнена разбивка по времени суток (утро/день/вечер/ночь):** результаты позволяют выявить *паттерны поведения* велосипедистов;
- **получены данные по конкретным дням:** рассчитаны значения для **среды** (типичный рабочий день) и **воскресенья** (выходной), которые демонстрируют существенные различия в характере поездок — рабочие дни характеризуются большим количеством поездок, когда спрос на передвижение по городу выше.

Также **создана интерактивная карта** с переключением временных слоёв. Размер и цвет маркеров отражают уровень активности станции, что обеспечивает **наглядную визуализацию** пространственного распределения поездок. Станции с нулевой активностью в конкретный период скрываются.

**Все результаты успешно рассчитаны, сохранены в CSV-файлы и визуализированы на интерактивной карте.**